In [5]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
   text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [7]:
# tokenize the chars
string_to_int = { ch: i for i, ch in enumerate(chars)} # create a dictionary of the characters
int_to_string = {i:ch for i, ch in enumerate(chars)} # create a reverse dictionary of the characters
encode = lambda words: [string_to_int[char] for char in words]
decode = lambda chars: ''.join([int_to_string[char] for char in chars])

encode_hello = encode('hello')
print(encode_hello)
decoded_hello = decode(encode_hello)
print(decoded_hello)


[61, 58, 65, 65, 68]
hello
